<a href="https://github.com/theonaunheim">
    <img style="border-radius: 100%; float: right;" src="static/strawberry_thief_square.png" width=10% alt="Theo Naunheim's Github">
</a>
<br style="clear: both">
<hr>
<br>


<div style="display: table; width: 100%">
    <div style="display: table-row; width: 100%;">
        <div style="display: table-cell; width: 50%; vertical-align: middle;">
            <img src="static/database.png" width="200">
        </div>
        <div style="display: table-cell; width: 10%">
        </div>
        <div style="display: table-cell; width: 40%; vertical-align: top;">
            <blockquote>
                <p style="font-style: italic;">"Syntax Error Near ' , '."</p>
                <br>
                <p>-Every RDBMS Ever</p>
            </blockquote>
        </div>
    </div>
</div>

<br>

<div align='left'>
    Image courtesy of <a href='https://commons.wikimedia.org/wiki/File:Applications-database.svg'>Dracos</a> under the <a href='https://creativecommons.org/licenses/by-sa/3.0/deed.en'>CC BY-SA 3.0</a>
</div>

<hr>

# Generally

Odds are you're going to need information from a database at some point. Python has got you covered. Here we're going to look through some of the more common libraries for querying, creating, and tinkering with databases.


---

# Modules covered

### Standard Library
* [csv](https://docs.python.org/3/library/csv.html)
* [getpass](https://docs.python.org/3.6/library/getpass.html)
* [os](https://docs.python.org/3/library/os.html)
* [sqlite3](https://docs.python.org/3/library/sqlite3.html)

### Third Party Libraries
* [pandas](https://pandas.pydata.org/pandas-docs/stable/)
* [pyodbc](https://github.com/mkleehammer/pyodbc/wiki)


# Modules not covered

### Standard Library
* None

### Third Party Libraries
* [sqlalchemy](https://www.sqlalchemy.org/)

---

In [ ]:
# Stdlib imports
import csv
import getpass
import os
import sqlite3

# Third party imports
import pandas as pd
import pyodbc

# Connecting to databases

### Note: you need to have the appropriate ODBC drivers and access permissions for these database queries to work properly. The examples below will not work unless you customize them for the particular database you are connecting to.

In [ ]:
print('{} has the following ODBC drivers available for PyODBC:\n'.format(os.environ['COMPUTERNAME']))
for driver in pyodbc.drivers():
    print(driver)

In [ ]:
# Usually it's easier to use arguments
CONNECTION_ARGS = {
    'driver': '{SQL Server Native Client 11.0}',
    'server': 'SERVER_ADDRESS',
    'database': 'DATABASE_NAME',
    'trusted_connection': 'yes',
    'readonly': True,
}

# But if you manually want to specify your connection string, you can.
CONNECTION_STRING = '''DRIVER={SQL Server Native Client 11.0};
                       SERVER=SERVER_ADDRESS;
                       DATABASE=DATABASE_NAME;
                       Trusted_connection=yes'''

QUERY = '''

SELECT TOP(5)
    COLUMN_1,
    COLUMN_2
FROM
    TABLE

'''

# Connection arguments allow us to open a connection like this:
conn = pyodbc.connect(**CONNECTION_ARGS)
# Then create a cursor.
cursor = conn.cursor()
# And run a query
cursor.execute(QUERY)
# And get results
rows = cursor.fetchmany(50)

# conn.commit()   # only if you need to commit a transaction
# conn.rollback() # only if you need to rollback a transaction
# Then we close our connection.
conn.close()

# Let's examine our results!
rows

In [ ]:
return_value_looks_like = [
    ('Wat1', 2.523),
    (None  , 5.523),
    ('Wat2', 2.123),
    ('Wat3', 2.123),
    ('Wat4', 4.523),
]

return_value_looks_like

### Again, this is way, way easier with a context manager (which automatically commits and closes):

In [ ]:
# Connecting with a string is also possible.
with pyodbc.connect(CONNECTION_STRING) as conn:
    # Create cursor
    cursor = conn.cursor()
    # Execute query
    cursor.execute(QUERY)
    # Fetch rows
    data = cursor.fetchall()
        
# Print fetched remaing rows
for row in data:
    print(row)

In [ ]:
for row in return_value_looks_like:
    print(row)

### Notice above we didn't have to enter our password because it used our active directory credentials with the "integrated_authentication=True" option. Oracle databases will other parameters such as a username, password, and dbq.

Reminders:

* never store your password on your computer.
* the password is stored as an unencrypted string unless you encrypt it.
* friends don't let friends use Oracle.

In [ ]:
# Lets get our credentials from the environment and enter our password
username = os.environ['USERNAME']
password = getpass.getpass('Enter your Oracle password here:')`

# Warning: getpass stores your password in cleartext in RAM.
# print(password)

In [ ]:
# For convenience, we will skip the connection string.
CONNECTION_ARGS = {
    'driver': '{Oracle in GeneralUser64_1}',
    'dbq': 'DBQ_NAME',
    'user': os.environ['USERNAME'],
    'pwd': password,
    'readonly': True,
}

# DAS QUERY
QUERY = '''

SELECT
    COLUMN_1,
    COLUMN_2
FROM
    TABLE
WHERE
    ROWNUM < 5

'''

In [ ]:
# The API is the same, and the datatypes should convert automatically.

# Open context manager
with pyodbc.connect(**CONNECTION_ARGS) as conn:
    # Create cursor
    cursor = conn.cursor()
    # Execute query
    cursor.execute(QUERY)
    # Fetch a single row
    first_row = cursor.fetchone()
    # Fetch all rows
    remaining_rows = cursor.fetchall()
        
# Print fetched remaing rows
for row in remaining_rows:
    print(row)

In [ ]:
for row in return_value_looks_like:
    print(row)

### If you're lazy like me, the absolute easist thing you can do is to load directly into a dataframe using the read_sql() method.

In [ ]:
with pyodbc.connect(**CONNECTION_ARGS) as conn:
    # read_sql() is the bee's knees.
    dialer_df = pd.read_sql(QUERY, conn)

dialer_df

In [ ]:
return_df_looks_like = pd.DataFrame(
    return_value_looks_like,
    columns=['COLUMN_1', 'COLUMN_2']
)

return_df_looks_like

### You also have access to most ODBC thing-a-ma-bobs.

In [ ]:
# Lets print out the available tables
with pyodbc.connect(**CONNECTION_ARGS) as conn:
    cursor = conn.cursor()
    table_list = list(cursor.tables())
    first_table = table_list[0][2]
    column_list = list(cursor.columns(table=first_table))

print('For table {}:'.format(first_table))

for column in column_list:
    print('\tColumn {} is of type {}.'.format(column[3], column[5]))

In [ ]:
print('Will look like:\n')

print('Table 1:')
print('\tColumn height is of type float.')
print('\tColumn active is of type bool.')
print('Table 2:')
print('\t... etc')

# SQLite

Sqlite3 is a small, efficient database that is everywhere. Odds are it is in your pocket on your smartphone right now. It is great for making a database on disk or in RAM.

Reminders:
* DO NOT STORE DATA THAT SHOULD BE ON A SERVER ON YOUR LOCAL MACHINE.

### You can create a database on your local file system simply by using the sqlite3.connect() command.

In [ ]:
BEETROOT_INVENTORY = [(5, 'red'), (3, 'purple'), (2, 'white')]

# Create a database in memory with the ':memory:' option.
conn = sqlite3.connect(':memory:')

# Create a cursor
cursor = conn.cursor()

# Create a table and populate with values from our list above.
cursor.execute('CREATE TABLE rutabagas(key INTEGER, pounds INTEGER, color TEXT, PRIMARY KEY(key DESC));')
cursor.executemany('INSERT INTO rutabagas VALUES(NULL, ?, ?)', BEETROOT_INVENTORY)

# Commit changes to database.
conn.commit()

# Select everything in our database and print it out.
cursor.execute('SELECT * FROM rutabagas')
print(cursor.fetchall())

# Connection closes, and database disappears
conn.close()

### More importantly, we can also write the DB to disk

Note: Again, context managers are the best way to do this, because they commit and close automatically.

In [ ]:
# Declare path to test database.
DB_PATH = './data/test.sqlite3' # often seen as test.db 

with sqlite3.connect(DB_PATH) as conn:
    # Create a cursor
    cursor = conn.cursor()
    # Create a table and populate with values from our list above.
    cursor.execute('CREATE TABLE IF NOT EXISTS rutabagas(key INTEGER, pounds INTEGER, color TEXT, PRIMARY KEY(key DESC));')
    cursor.executemany('INSERT INTO rutabagas VALUES(NULL, ?, ?)', BEETROOT_INVENTORY)
    # Implicit commit and close.

### As usual, things are generally better with pandas ...

In [ ]:
# Read our CSV into a dataframe
df = pd.read_csv('./data/iris_dataset.csv')

# Open SQLite connection to write
with sqlite3.connect('./data/test.sqlite3') as conn:
    # Write to database, which allows you to add lots of files to a single source.
    df.to_sql('iris_table', conn, if_exists='replace', index=False)

In [ ]:
# Read to prove we wrote it
with sqlite3.connect('./data/test.sqlite3') as conn:
    # Same as before.
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM iris_table LIMIT 5;')
    records = cursor.fetchall()

records

# Additional Learing Resources

* ### [SQLite3.org](https://www.sqlite.org/index.html)


---

# Next Up: [Files](4_files.ipynb)


<img style="margin-left: 0;" src="static/file.png" width="100">

---